# Word Search Suggestion System

This project is a prototype of a search suggestion system, inspired by how modern search engines provide real-time recommendations as users type. The system simulates the behavior of autocomplete features commonly seen in search engines like Google, where the most relevant or frequently selected terms appear at the top of the suggestion list.

## The primary goal is to build a system that:

- Efficiently retrieves all words that begin with a given prefix.
- Suggests alternative or similar words when an exact prefix match isn't found.
- Sorts suggestions based on how frequently they have been selected in the past.

The prototype mimics this behavior using a simplified dataset of words and tracks how often each word is chosen, enabling dynamic and frequency-based sorting of results.

## Data Structures

- Trie (Prefix Tree):
A trie is used to store the list of words in a structured way that allows fast retrieval based on prefixes. This ensures that, given a search string, all matching words can be found efficiently by traversing down the trie.

- HashMap (Dictionary):
A dictionary (searchedWords) is used to track how many times each word has been selected. This enables the system to sort and prioritize frequently chosen words in future search results.

## Algorithms

- Depth-First Search (DFS):
DFS is used to recursively traverse the trie and collect all words that share a given prefix. This is essential when we need to suggest all possible completions for a user's input.

- Quick Sort:
Once potential suggestions are found, they are sorted based on their frequency using Quick Sort. This helps ensure that more popular search terms appear earlier in the list of recommendations.




## TrieNode

The TrieNode class is the fundamental building block of a Trie data structure. It defines the structure and attributes of each node within the Trie.

### Each node contains the following attributes:

- value : Represents the character stored at the current node. It typically holds a character from the set [a-zA-Z0-9], but can also support special characters depending on the use case.

- children : A dictionary that maps each child character to its corresponding TrieNode. This allows dynamic branching for every possible continuation of the prefix.

- isEnd : A boolean flag that indicates whether this node marks the end of a valid word in the Trie. This is essential for distinguishing complete words from prefixes.

In [4]:
class TrieNode:
    def __init__(self, value):
        self.value = value
        self.children = {}
        self.isEnd = False

    def __str__(self):
        return f"value : {self.value}, children : {list(self.children.keys())}"

## Trie

The Trie class provides the complete implementation of a Trie (Prefix Tree)—a tree-like data structure optimized for fast retrieval of strings, particularly useful for tasks such as autocomplete, spell checking, and prefix-based searching.

The Trie starts with an empty root node (startNode), represented by a TrieNode initialized with a space character " ". All words inserted into the Trie branch out from this root node via its children dictionary.

### Methods

#### __init__()

Initializes the Trie with a root node. This node does not hold any character from an actual word but acts as the starting point for all insertions and searches.

#### insert(word: str) -> None : 
Inserts a word into the Trie

- Iterates over each character in the word.
- If the character already exists in the current node's children, it moves to that node.
- If not, it creates a new TrieNode and adds it to the children.
- After inserting all characters, it marks the final node's isEnd flag as True to indicate the end of a valid word.

#### search(word: str) -> bool : 
Searches for a complete word in the Trie. This method is not used in the prototype but its part of the implementation

- Traverses the Trie character by character.
- Returns False if any character is not found.
- Returns True only if the traversal ends at a node where isEnd is True, confirming it's a complete word in the Trie.

#### startsWith(prefix: str) -> object
Checks if any word in the Trie starts with the given prefix.

- Traverses the Trie through the given prefix.
- Returns the node where the prefix traversal ends if it exists.
- Returns None if the prefix is not found.

This method is especially useful for retrieving all words that share a common prefix, enabling powerful features like autocompletion.



In [5]:
class Trie:
    def __init__(self):
        self.startNode = TrieNode(" ")

    def insert(self, word: str) -> None:
        currentNode = self.startNode
        for c in word:
            if c in currentNode.children:
                currentNode = currentNode.children[c]
            else:
                node = TrieNode(c)
                currentNode.children[c] = node
                currentNode = node
        
        currentNode.isEnd = True

    def search(self, word: str) -> bool:
        currentNode = self.startNode
        for c in word:
            if c in currentNode.children:
                currentNode = currentNode.children[c]
            else:
                return False

        return currentNode.isEnd

    def startsWith(self, prefix: str) -> object:
        currentNode = self.startNode
        for c in prefix:
            if c in currentNode.children:
                currentNode = currentNode.children[c]
            else:
                return None
        
        return currentNode


## Helper methods

These utility functions support the core functionality of the Word Search Suggestion System. They handle tasks such as traversing the trie, sorting, and checking character relationships—ensuring that suggestions are relevant, correctly ordered, and efficiently retrieved.

---

### dfs(node: object, string: str, result: list) -> None
Performs a Depth-First Search starting from a given Trie node to collect all complete words beneath that node.

- Parameters:

    - node: The current TrieNode being explored.
    - string: The current prefix string formed so far.
    - result: A list to collect completed words.

- Behavior:

    - If the node marks the end of a word, the accumulated string is added to result.
    - Recursively explores each child node by appending its character to the string.

Used when generating suggestions based on a prefix.

---

### sort_chars_lex(s: str) -> str
Sorts the characters of a string in lexicographical order.

- Example:

    - Input: "wolf"
    - Output: "flow"

This helps normalize strings for comparison in similarity checks (e.g., when looking for potential matches if a prefix isn't found directly in the trie).

---

### is_subsequence(small: str, large: str) -> bool
Checks whether the string small is a subsequence of the string large.

Uses an iterator over large and verifies that all characters of small appear in order within large.

- Example:

    - is_subsequence("flow", "flower") → True
    - is_subsequence("flw", "wolf") → False

Used to suggest words that may not exactly match the input but contain its characters in the same order.

---

### quicksort_dict_by_value(d: dict) -> list
Sorts a dictionary by its values using the Quicksort algorithm.

- Parameters:

    - d: A dictionary where keys are words and values are their corresponding search counts.

- Returns:

    - A sorted list of key-value tuples in descending order of frequency.

This is critical for ranking search suggestions by popularity.



In [11]:
def dfs(node : object, string : str, result : list) -> None:
    if node.isEnd:
        result.append(string)
                
    if len(node.children) == 0:
        return

    for child in node.children.values():
        dfs(child, string + child.value, result)

def sort_chars_lex(s: str) -> str:
    return "".join(sorted(s))

def is_subsequence(small: str, large: str) -> bool:
    it = iter(large)
    return all(ch in it for ch in small)

def quicksort_dict_by_value(d: dict) -> list:
    if len(d) <= 1:
        return list(d.items())
    
    items = list(d.items())
    pivot = items[0]
    rest = items[1:]

    greater = [
        item for item in rest
        if item[1] > pivot[1] or (item[1] == pivot[1] and item[0] < pivot[0])
    ]
    lesser = [
        item for item in rest
        if item[1] < pivot[1] or (item[1] == pivot[1] and item[0] >= pivot[0])
    ]

    return quicksort_dict_by_value(dict(greater)) + [pivot] + quicksort_dict_by_value(dict(lesser))

## Inserting words into the Trie

The program reads words from a file named words.txt, stripping any whitespace and storing valid entries in a list. It then initializes a Trie and inserts each word into it, character by character. This builds an efficient prefix tree structure, enabling fast word lookups and autocomplete suggestions based on user input.

In [7]:
words = []
#reading the words from the words.txt and storing it in words list
with open('words.txt', 'r', encoding='utf-8') as f:
    for line in f:
        word = line.strip()          
        if word:                    
            words.append(word)

trie = Trie()

#inserting the words into the trie
for word in words:
    trie.insert(word)

## Word Search

The wordSearch function powers the core suggestion logic of the system. Given a user-input string, it searches for matching words in the trie. If exact matches are found (using the prefix), it gathers all possible completions using a depth-first search. If no matches are found, it tries to suggest similar words by checking if the input is a character-wise subsequence of other words that start with the same letter.

It then ranks the results based on how frequently each word has been searched, using a quicksort-based sorting algorithm. The top 10 suggestions are displayed, and the user can choose one. The selected word's search count is then updated in the searchedWords dictionary. This makes the system adaptive, promoting frequently chosen words to the top of future suggestions.

In [117]:
import random

def wordSearch(searchString : str, searchedWords : dict, test = False) -> object:
    
    # to collect the similar words if the searchedString is not present in the words
    result = []
    currentWordsCount = {}

    node = trie.startsWith(searchString)

    if not node:
        sortedSearchString = sort_chars_lex(searchString)
        allwords = []
        dfs(trie.startNode, "", allwords)
        # add words with same starting character as of inputString to a list
        for word in allwords:
            if word[0] == searchString[0]:
                sortedWord = sort_chars_lex(word)
                #checking whether the searchedString is a subsequence of the words
                if is_subsequence(sortedSearchString, sortedWord):
                    result.append(word)

        if len(result) > 0:
            print("Did you mean?")
            
        else:
            print("Enter a valid word... ")
            return
    
    else:
        #recursively find all the strings from that common node child
        dfs(node, searchString, result)

    for word in result:
        if word not in searchedWords:
            currentWordsCount[word] = 0
        else:
            currentWordsCount[word] = searchedWords[word]

    currentWordsCount = quicksort_dict_by_value(currentWordsCount)
    currentWordsCount = dict(currentWordsCount[0:10])

    choice = 0

    if not test:
        if(len(currentWordsCount) == 1):
            choice = 1

        else:
            print("Choose a word from the below list you want to search (1 for the 1st word)")
            print(list(currentWordsCount.keys()))
            
            choice = int(input())
    else:
        print("Choose a word from the below list you want to search (1 for the 1st word)")
        print(list(currentWordsCount.keys()))
        choice = random.randint(0, len(currentWordsCount))
        
    searchedWords[str(list(currentWordsCount.items())[choice - 1][0])] = list(currentWordsCount.items())[choice - 1][1] + 1

    print(f"Searched Words count {searchedWords}")

    

## Test Scenario

Assume we are simulating a search engine. Each time the user types a character, the function wordSearch() is invoked, simulating a search suggestion system. For each invocation, the function returns a list of up to 10 recommended words based on the current input (i.e., the searchString). These suggestions are ordered by how frequently each word has been selected in the past, mimicking how real search engines prioritize popular searches.

If the exact word is found in the trie, we retrieve all words that begin with that prefix. Otherwise, we look for words that:

- Start with the same character as the input.
- Contain all characters of the input in sorted order (i.e., they are an anagram or supersequence).

This allows us to suggest similar alternatives when the exact word is not present.

From the list of suggestions, the user selects one, and we update its search count in the searchedWords dictionary. Over time, more frequently selected words will rise to the top of the suggestion list.

## Automated Testing Behavior
For testing purposes (test=True), the selection is automated: a random word from the suggestions is picked instead of prompting for user input. By repeatedly running the function with the same searchString, we can observe how the search counts evolve. As certain words are selected more often, they will appear higher in the list, demonstrating the effect of frequency-based sorting.

searchString = "flow" is used for the test. There are only 2 words that are prefixed with "flow" i.e, "flower" and "flow". At the start, both counts are 0 and as a random word is chosen, the count starts to increase and the effect of frequecy-based-sorting can be observed.

## Test scenarios

#### Scenario 1: The searchString exist the in Trie

    Input: searchString = "flow"

    - Call 1 : 
    Searched Words count {}
    Suggestions: ['flow', 'flower']
    User picks "flow"
    Searched Words count {'flow': 1}
    
    - Call 2
    Suggestions: ['flow', 'flower'] (equal frequency, still alphabetical)
    User picks "flower"
    Searched Words count {'flow': 1, 'flower': 1}
    
    - Call 3
    Suggestions: ['flow', 'flower'] (still equal frequency)
    User picks "flower"
    Searched Words count {'flow': 1, 'flower': 2}
    
    - Call 4
    Suggestions: ['flower', 'flow']
    "flower" now has higher frequency, hence appears first
    User picks "flower"
    Searched Words count {'flow': 1, 'flower': 3}



#### Scenario 2: The searchString does not exist in the Trie

    Input: searchString = "ck"

    Since no word in the Trie starts with "ck", the function searches for similar words.
    - Call 1
    Searched Words count {}
    Suggestions: ['chalk', 'check', 'cheek', 'chicken', 'chunk', 'clock']
    (All have 0 count → alphabetical order)
    User picks 'chicken'
    Searched Words count {'chicken': 1}
    
    - Subsequent Calls
    As different words are picked, the count increases, and higher-frequency words begin appearing at the top of the list.


In [196]:
searchedWords = {}

In [208]:
wordSearch("cho", searchedWords, True)

Choose a word from the below list you want to search (1 for the 1st word)
['chorus', 'choice', 'choose']
Searched Words count {'chorus': 7, 'choose': 2, 'choice': 3}


## Actual scenario

The program simulates a real-time search suggestion experience. It continuously prompts the user to enter a word or prefix, then calls the wordSearch function to display relevant suggestions from the Trie. The user can interactively choose one of the suggested words, and the system updates the search frequency accordingly in the searchedWords dictionary.

This loop continues until the user chooses to stop by entering "y" when prompted. Over time, frequently selected words rise to the top of the suggestions, mimicking a search engine’s adaptive behavior.

In [10]:
searchedWords = {}
while True:
    print("-------------------------------------------------------------------------------------")
    searchString = input("Enter a word to see all the words that starts with it : ")
    
    wordSearch(searchString, searchedWords)

    end = input("Stop searching? y or n: ").strip().lower()
    if end == "y":
        break

-------------------------------------------------------------------------------------


Enter a word to see all the words that starts with it :  flow



Choose a word from the below list you want to search (1 for the 1st word)
['flower', 'flow']


 2


Searched Words count {'flow': 1}


...

Enter a word to see all the words that starts with it :  flo



Choose a word from the below list you want to search (1 for the 1st word)
['flower', 'flow', 'flock']


 2


Searched Words count {'flow': 2, 'flower': 2}


Stop searching? y or n:  y
